# Reinforcement Learning - Lab 4 (graded)
### J. Martinet

Implement Q-learning from scratch

Duration: 90 min

### 1) First version with a 1D grid world


We have discussed Q-learning during the class. As you know, it is an off-policy algorithm that uses the Time Difference $\delta_t$, which is the difference between the estimated value of $s_t$ and the better estimate $r_{t+1} + \gamma V^\pi (s_{t+1})$

$$ \delta_t = r_{t+1} + \gamma V^\pi (s_{t+1}) - V^\pi (s_t) $$

The general definition of Q-learning update rule is:

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha[ r_{t+1} + \gamma \max_a Q(s_{t+1}, a) - Q(s_t,a_t) ] $$


In this part, we are going to implement Q-learning in the simple setting of a 1D grid world:

![1D grid world](RL4_1dgrid.png)

Make sure you understand:
- the size of the grid world (= number of states)
- the size of the action space (= number of possible actions)
- the size of the Q-table
- the expected reward for reaching each state

The first step will be to initialize an empty Q-table, a table of rewards, a move cost, and alpha and gamma parameters.

In [ ]:
import numpy as np
from random import random, randint

# we have 2 actions : move left and move right
nb_action = 2
nb_state = 6

# we create a matrix 6*2 to represent the value of each move at a given state
QTable = np.zeros((nb_state,nb_action))

# the tab with the representation of the 6 states (-1 for the bad end, 1 for the good end, and 0 for other states)
REWARD = [-1,0,0,0,0,1 ]

# cost of one move
cost = 0.01

# learning rate - should not be too high, e.g. between .5 and .9
alpha = 0.9

# discount factor that shows how much you care about future (remember 0 for myopic)
gamma = 0.5

Now comes the interesting part. You need to write the main Q-learning loop.

The first version will simply iterate:
- choose an action (by looking up in the Q-table! Choose the most interesting move)
- move
- update the Q-table

When you get this version, you can make it more complete to add the exploration/exploitation with the $\epsilon$-greedy version, by initializing an $\epsilon = 1$ that you decrease by e.g. 0.01 in each iteration.

In your main loop, start by drawing a random number. If it is lower that $\epsilon$, then EXPLORE (= take a random move), otherwise EXPLOIT (= choose the best move)

In [ ]:
# your code here
EPSILON_START = 1
EPSILON_DECAY = 0.01
EPISODES = 100

def step(state, QTable, epsilon):
    if random() < epsilon:
        action_idx = randint(0, nb_action - 1)
    else:
        action_idx = np.argmax(QTable[state])

    if action_idx == 0:
        action = -1
    else:
        action = 1

    next_state = state + action
    next_state = max(0, min(next_state, nb_state - 1))

    act_reward = REWARD[next_state] - cost

    isTerminated = False
    if REWARD[next_state] == -1 or REWARD[next_state] == 1:
        isTerminated = True

    return action_idx, next_state, act_reward, isTerminated

def qLearning(QTable):
    epsilon = EPSILON_START
    for episode in range(EPISODES):
        state = 2

        i = 1
        while True:
            # print(f"Episode {episode}: iteration {i}")
            action_idx, next_state, act_reward, isTerminated = step(state, QTable, epsilon)
            old_val = QTable[state][action_idx]
            next_max = max(QTable[next_state])

            new_val = old_val + alpha * (act_reward + gamma * next_max - old_val)
            QTable[state][action_idx] = new_val

            state = next_state

            i += 1

            if isTerminated:
                break
                
        epsilon -= EPSILON_DECAY
        epsilon = max(0, epsilon)

    return QTable
    
def main():
    Q = qLearning(QTable)

    print("\nOutput legend:")
    print("State | Best action")
    actions = ["L", "R"]
    for s in range(nb_state):
        best = np.argmax(Q[s])
        print(f" {s} | {actions[best]} |")

main()



### 2) Second version with a 2D grid world

Same exercise, in the following 2D grid:

![2D grid world](RL4_2dgrid.png)

In [12]:
# your code here
import numpy as np
from random import random, randint

ROWS = 3
COLS = 4
nb_action = 4

QTable = np.zeros((ROWS, COLS, nb_action))
WALL = (1, 1)

REWARD_MAP = np.zeros((ROWS, COLS))
REWARD_MAP[0, 3] = 1   
REWARD_MAP[1, 3] = -1 

cost = 0.01
alpha = 0.9
gamma = 0.5

EPSILON_START = 1
EPSILON_DECAY = 0.01
EPISODES = 100

def step(state, QTable, epsilon):
    row, col = state

    if random() < epsilon:
        action_idx = randint(0, nb_action - 1)
    else:
        action_idx = np.argmax(QTable[row][col])

    actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    move_r, move_c = actions[action_idx]

    next_r = row + move_r
    next_c = col + move_c
    next_r = max(0, min(next_r, ROWS - 1))
    next_c = max(0, min(next_c, COLS - 1))

    if (next_r, next_c) == WALL:
        next_r = row
        next_c = col

    next_state = (next_r, next_c)

    act_reward = REWARD_MAP[next_r][next_c] - cost

    isTerminated = False
    if REWARD_MAP[next_r][next_c] == -1 or REWARD_MAP[next_r][next_c] == 1:
        isTerminated = True

    return action_idx, next_state, act_reward, isTerminated

def qLearning(QTable):
    epsilon = EPSILON_START
    for episode in range(EPISODES):
        state = (2, 0)

        i = 1
        while True:
            # print(f"Episode {episode}: iteration {i}")
            action_idx, next_state, act_reward, isTerminated = step(state, QTable, epsilon)
            r = state[0]
            c = state[1]
            next_r = next_state[0]
            next_c = next_state[1]

            old_val = QTable[r][c][action_idx]
            next_max = max(QTable[next_r][next_c])

            new_val = old_val + alpha * (act_reward + gamma * next_max - old_val)
            QTable[r][c][action_idx] = new_val

            state = next_state

            i += 1
            
            if isTerminated:
                break
            
        epsilon -= EPSILON_DECAY
        epsilon = max(0, epsilon) 

    return QTable

def main():
    Q =qLearning(QTable)
    print("\nOutput Legend:")
    print("U: Up, D: Down, L: Left, R: Right")

    moves_symbols = {0: "U", 1: "D", 2: "L", 3: "R"}

    for r in range(ROWS):
        row_str = "|"
        for c in range(COLS):
            if (r, c) == WALL:
                row_str += " WALL |"
            elif (r, c) == (0, 3): # Goal
                row_str += " GOAL |"
            elif (r, c) == (1, 3): # Trap
                row_str += " TRAP |"
            else:
                best_action = np.argmax(Q[r][c])
                row_str += f"  {moves_symbols[best_action]} |"
        print(row_str)

main()


Output
|  R |  R |  R | GOAL |
|  U | WALL |  U | TRAP |
|  U |  R |  U |  L |


### 3) Optional third part (with bonus): plot the evolution of the total reward

Make a plot of the evolution of the total reward after each epidode during the simulation / learning with different values of $\gamma$, $\alpha$, and $\epsilon$.

In [13]:
# your code here
import numpy as np
from random import random, randint
import matplotlib.pyplot as plt

ROWS = 3
COLS = 4
nb_action = 4
WALL = (1, 1)

REWARD_MAP = np.zeros((ROWS, COLS))
REWARD_MAP[0, 3] = 1 
REWARD_MAP[1, 3] = -1

cost = 0.01

def step(state, QTable, epsilon):
    row, col = state

    if random() < epsilon:
        action_idx = randint(0, nb_action - 1)
    else:
        action_idx = np.argmax(QTable[row][col])

    actions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    move_r, move_c = actions[action_idx]

    next_r = row + move_r
    next_c = col + move_c

    next_r = max(0, min(next_r, ROWS - 1))
    next_c = max(0, min(next_c, COLS - 1))

    if (next_r, next_c) == WALL:
        next_r = row
        next_c = col

    next_state = (next_r, next_c)

    act_reward = REWARD_MAP[next_r][next_c] - cost

    isTerminated = False
    if REWARD_MAP[next_r][next_c] == -1 or REWARD_MAP[next_r][next_c] == 1:
        isTerminated = True

    return action_idx, next_state, act_reward, isTerminated

def run(alpha, gamma, epsilon_start, epsilon_decay, episodes):
    QTable = np.zeros((ROWS, COLS, nb_action))ù
    
    
    